### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
import numpy as np

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [ ]:
print(tokenizer.decode(qqp_preprocessed['train'][0]['input_ids'])[:200])
print(qqp_preprocessed['train'][0]['input_ids'][:50])

### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=3, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0, 0, 1]), 'idx': tensor([0, 1, 2]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for batch in val_loader:
     break  # here be your training code
# print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids'],
  )

# print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

predict_t = torch.tensor([], device='cuda')
labels = torch.tensor([], device='cuda')



predict_t = torch.stack((*list(predict_t), *list(torch.argmax(torch.softmax(predicted.logits, dim=1).data, dim=-1))))
predict_t = torch.stack((*list(predict_t), *list(torch.argmax(torch.softmax(predicted.logits, dim=1).data, dim=-1))))

labels = torch.stack((*list(labels), *list(batch['labels'])))
labels = torch.stack((*list(labels), *list(batch['labels'])))
accuracy_score(labels, predict_t)

In [ ]:
model.to('cuda')

val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=64, shuffle=False, collate_fn=transformers.default_data_collator,
    num_workers=2
)

predict_t = torch.tensor([], device='cuda')
labels = torch.tensor([], device='cuda')

for batch in val_loader:
    # print("Sample batch:", batch)
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    token_type_ids = batch['token_type_ids'].to('cuda')
    label_batch = batch['labels'].to('cuda')

    with torch.no_grad():
        predicted = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids
        )
        # print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())
        predictions = torch.argmax(torch.softmax(predicted.logits, dim=1), dim=-1)

    predict_t = torch.cat((predict_t, predictions), dim=0)
    labels = torch.cat((labels, label_batch), dim=0)

In [ ]:
predict_t = predict_t.to('cpu')
labels = labels.to('cpu')

accuracy = accuracy_score(labels, predict_t)
accuracy

0.9083848627256987

In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
model = transformers.AutoModel.from_pretrained('microsoft/deberta-v3-base', num_labels=2)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

print(tokenizer.decode(qqp_preprocessed['train'][0]['input_ids'])[:200])
print(qqp_preprocessed['train'][0]['input_ids'][:50])

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

[CLS] How is the life of a math student? Could you describe your own experiences?[SEP] Which level of prepration is enough for the exam jlpt5?[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PA
[1, 577, 269, 262, 432, 265, 266, 5291, 1234, 302, 5047, 274, 3443, 290, 451, 2056, 302, 2, 2597, 674, 265, 9649, 24479, 269, 618, 270, 262, 3086, 4402, 24580, 297, 524, 302, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
      super().__init__()
      self.bert =  transformers.AutoModel.from_pretrained('microsoft/deberta-v3-base')
      self.pooler = nn.Linear(768,768)
      self.activation = torch.tanh
      # print(self.bert.pooler)
      self.out = nn.Linear(768,2)


    def forward(self, input_ids, attention_mask, token_type_ids, labels, **kwargs):
      # states = self.bert(input_ids, **kwargs).last_hidden_state
      states = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            # position_ids=position_ids,
            # output_hidden_states=output_hidden_states
        ).last_hidden_state
      # print(states.last_hidden_state.shape)
      cls_emb = states[:,0,:]
      logits = self.activation(self.pooler(cls_emb))
      return self.out(logits)

    # def forward(self, input_ids, attention_mask, token_type_ids, position_ids, labels, output_hidden_states=True):

    #     states = self.bert(
    #         input_ids,
    #         attention_mask=attention_mask,
    #         token_type_ids=token_type_ids,
    #         position_ids=position_ids,
    #         output_hidden_states=output_hidden_states
    #     )


    #     cls_emb = states[:,0,:]
    #     logits = self.activation(self.pooler(cls_emb))
    #     logits = self.out(logits)
          # probs = torch.softmax(logits, dim=-1)

    #     loss = nn.CrossEntropyLoss(probs, labels)


    #     return transformers.BertModelOutput(
    #         loss=loss,
    #         logits=logits,
    #         hidden_states=outputs.hidden_states,
    #         attentions=outputs.attentions,
    #     )

In [ ]:
model = BertClassifier()

In [ ]:
MAX_LENGTH = 128
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding='max_length', max_length=MAX_LENGTH, truncation=True, return_tensors="pt")
tokens_info

{'input_ids': tensor([[   1, 6201,  261,  273,  481,  290, 1503,  260,    2,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   1, 1982,  269,  339, 2005,  335,  274,  280,  368, 2418,  570,  340,
         1254,  260,    2,    0,    0

In [ ]:
with torch.no_grad():
  out = model(**tokens_info)

out

tensor([[-0.0047,  0.0607],
        [ 0.0030,  0.0567]])

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=3, shuffle=False, collate_fn=transformers.default_data_collator
)

for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch["labels"])

# with torch.no_grad():
#   predicted = model(
#       input_ids=batch['input_ids'],
#       attention_mask=batch['attention_mask'],
#       token_type_ids=batch['token_type_ids'],
#   )
with torch.no_grad():
    predicted = model(**batch)


print(predicted)
# print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: tensor([0, 0, 1])
tensor([[0.0621, 0.1913],
        [0.0652, 0.1873],
        [0.0675, 0.1966]])


In [ ]:
batch_size=5

train_set = qqp_preprocessed['train']
val_set = qqp_preprocessed['validation']


train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, collate_fn=transformers.default_data_collator,
    num_workers=2
)

val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=True, collate_fn=transformers.default_data_collator,
    num_workers=2
)


In [ ]:
def calculate_loss(logits, labels):
    # probs = torch.softmax(logits, dim=-1)
    loss = nn.CrossEntropyLoss()
    return loss(logits, labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(logits, labels):
    probs = torch.softmax(logits, dim=-1)
    preds = probs.argmax(dim = -1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"acc":acc, "f1":f1}

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from tqdm import tqdm, trange


metrics = {'train_loss': [], "accuracy":[], "f1":[]}
opt = torch.optim.Adam(model.parameters())
batch_size = 3

In [ ]:
for _ in trange(50):
    step = len(metrics['train_loss']) + 1
    data_inp = next(iter(train_loader))
    out_logits = model(**data_inp)
    labels = data_inp['labels']
    loss_t = calculate_loss(out_logits, labels)
    print(loss_t)
    opt.zero_grad()
    loss_t.backward()
    opt.step()



    metrics['train_loss'].append((step, loss_t.item()))

    if step % 2 == 0:
        m = compute_metrics(out_logits, labels)
        metrics['accuracy'].append((step, m['acc']))
        metrics['f1'].append((step, m['f1']))

        clear_output(True)
        plt.figure(figsize=(12,4))
        for i, (name, history) in enumerate(sorted(metrics.items())):
            plt.subplot(1, len(metrics), i + 1)
            plt.title(name)
            plt.plot(*zip(*history))
            plt.grid()
        plt.show()
        print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-10:], axis=0)[1], flush=True)

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(0.6730, grad_fn=<NllLossBackward0>)


  0%|          | 0/50 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments


tokenizer = transformers.AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
MAX_LENGTH = 128
batch_size = 32

def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

print(tokenizer.decode(qqp_preprocessed['train'][0]['input_ids'])[:200])
print(qqp_preprocessed['train'][0]['input_ids'][:50])

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

[CLS] How is the life of a math student? Could you describe your own experiences?[SEP] Which level of prepration is enough for the exam jlpt5?[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PA
[1, 577, 269, 262, 432, 265, 266, 5291, 1234, 302, 5047, 274, 3443, 290, 451, 2056, 302, 2, 2597, 674, 265, 9649, 24479, 269, 618, 270, 262, 3086, 4402, 24580, 297, 524, 302, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(metrics)

{'train_loss': [(1, 0.7549327611923218), (2, 1.716436743736267), (3, 0.6319792866706848), (4, 1.1927257776260376), (5, 0.18677467107772827), (6, 0.8739132881164551), (7, 0.50400310754776), (8, 0.5004048347473145), (9, 0.2665655016899109), (10, 0.5004090070724487), (11, 1.4300177097320557), (12, 1.1339492797851562), (13, 0.6602791547775269), (14, 0.7531658411026001), (15, 0.9229680299758911), (16, 0.8237478137016296), (17, 0.6650265455245972), (18, 0.6798360347747803), (19, 0.5095944404602051), (20, 1.0728497505187988), (21, 0.5015658736228943), (22, 1.1252244710922241), (23, 0.5004149675369263), (24, 0.986151397228241), (25, 0.8727671504020691), (26, 0.6730268001556396), (27, 0.6929837465286255), (28, 0.8515685796737671), (29, 0.7856358885765076), (30, 0.773406445980072), (31, 0.8011006116867065), (32, 0.6621779203414917), (33, 0.6953674554824829), (34, 0.7371516823768616), (35, 1.091858983039856), (36, 0.8418272733688354), (37, 0.5095076560974121), (38, 1.00010085105896), (39, 0.71399

In [ ]:
training_args = TrainingArguments(
     output_dir='ouputs/',
     num_train_epochs = 3.0,
     per_device_train_batch_size = 32,
     per_device_eval_batch_size = 32,
     learning_rate=1e-5,
     fp16=True
)


trainer = Trainer(
    model,
    training_args,
    # compute_metrics=compute_metrics,
    train_dataset=qqp_preprocessed["train"],
    eval_dataset=qqp_preprocessed["validation"],
    data_collator=transformers.default_data_collator,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-5-bedcd22ebb52>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2024-10-25 17:40:53,631] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.419300
1000,0.312900
1500,0.296500
2000,0.278300
2500,0.274700
3000,0.261000
3500,0.261400
4000,0.267900
4500,0.254600
5000,0.248700


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainer.save_model('/content/drive/MyDrive/deberta-v3-base1')

In [ ]:
import shutil
import os

# Specify the source file path and destination directory path
source_path = "/content/ouputs/checkpoint-15000/optimizer.pt"
destination_path = "/content/drive/MyDrive/deberta-v3-base/15k/"  # Include the filename to copy
print(os.listdir(destination_path))

# Copy the file
shutil.copy(source_path, destination_path)

['model.safetensors']


'/content/drive/MyDrive/deberta-v3-base/15k/optimizer.pt'

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

model_path = '/content/drive/MyDrive/debertablya'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

In [ ]:
model.to('cuda')

val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=64, shuffle=False, collate_fn=transformers.default_data_collator,
    num_workers=2
)

predict_t = torch.tensor([], device='cuda')
labels = torch.tensor([], device='cuda')

for batch in val_loader:
    # print("Sample batch:", batch)
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    token_type_ids = batch['token_type_ids'].to('cuda')
    label_batch = batch['labels'].to('cuda')

    with torch.no_grad():
        predicted = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids
        )
        # print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())
        predictions = torch.argmax(torch.softmax(predicted.logits, dim=1), dim=-1)

    predict_t = torch.cat((predict_t, predictions), dim=0)
    labels = torch.cat((labels, label_batch), dim=0)

In [ ]:
from sklearn.metrics import accuracy_score
predict_t = predict_t.to('cpu')
labels = labels.to('cpu')

accuracy = accuracy_score(labels, predict_t)
accuracy

0.9131585456344299

### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

train_set = qqp_preprocessed['train']['text2']
# print(train_set)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=128, shuffle=False,
    num_workers=2
)


def k_nearest_questions(questions: list, k: int = 5) -> list:
    # question_tokens1 = tokenizer(questions[0], qqp_preprocessed['train']['text2'][1], padding=True, truncation=True, return_tensors="pt").to(device)
    # n_rows = len(qqp_preprocessed['train']['text'])
    result = []
    for q in questions:

        dup_scores = torch.tensor([], device=device)

        for batch in train_loader:
            # print("Sample batch:", batch)

            n_rows = len(batch)

            with torch.no_grad():
                # predicted = model(
                #     input_ids = input_ids,
                #     attention_mask = attention_mask,
                #     token_type_ids = token_type_ids
                # )
                question_tokens = tokenizer([q]*n_rows, batch, padding=True, truncation=True, return_tensors="pt").to(device)
                # print(question_tokens)

                model_outputs = model(**question_tokens)['logits']
                # print(model_outputs)

                dup_scores = torch.cat([dup_scores, model_outputs[:,1]])
                # indices = np.argpartition(dup_scores, -k)[-k:]

                # sorted_indices = indices[np.argsort(-dup_scores[indices])]

        # print('dup scores', dup_scores)

        dup_topk = torch.topk(dup_scores, k)
        # print('dup top k', dup_topk)
        idx = dup_topk.indices

        duplicates = [qqp_preprocessed['train']['text2'][i] for i in idx]
        # print('dups', duplicates)

        result.append(tuple(zip(duplicates, dup_topk.values.cpu().numpy())))


    return result

    # print(question_tokens2)
    # print(qqp_preprocessed['train']['input_ids'][:3])

    # print(tokenizer.decode(question_tokens))
    # labels1 = model(**question_tokens1)
    # labels2 = model(**question_tokens2)
    # print(labels1)
    # print(labels2)

In [ ]:
question = ['How old is the universe?',
            'Who is the first president of Armenia?',
            'What are the most surreal places to visit?',
            'When does Quora suck balls?',
            '"Can u breath undewater if u use nosplug but also eat banan? Asking for a friend."']

question1 = ["Who first president Armenia"]
question2 = ["Who is the first president of Armenia?"]


# train_set = qqp_preprocessed['train']['text2'][:20]

# train_loader = torch.utils.data.DataLoader(
#     train_set, batch_size=10, shuffle=False,
#     num_workers=2
# )

# for batch in train_loader:
#   print(batch)

# q = question[0]
# n_rows = len(qqp_preprocessed['train']['text2'][:10])

# print(np.tile(q, (n_rows, 1)).shape)
# print(question1*n_rows)
# print(qqp_preprocessed['train']['text2'])
# print(tokenizer(question1*n_rows, qqp_preprocessed['train']['text2'][:10], padding=True, truncation=True, return_tensors="pt").to(device))
# print(tokenizer.decode(qqp_preprocessed['train']['input_ids'][1]))

k_nearest_questions([question[0]], k=5)

In [ ]:
x = torch.arange(1., 6., device='cuda')
res = torch.topk(x, 3)
print(res.values)

idx = res.indices.cpu().numpy()
x[idx]


tensor([5., 4., 3.], device='cuda:0')


tensor([5., 4., 3.], device='cuda:0')